# Book 4 - Modelo XGBoost

Este notebook serviu como registro prático e teórico no meu aprendizado de Machine Learning.

`Enriqueci este notebook com anotações adicionais e aplicações práticas tornando-o uma referência valiosa para consultas e implementações em futuros projetos reais.`

Espero que este material inspire outros a explorar ainda mais o fascinante mundo do Machine Learning. 





Tópicos abordados:

- **Validação Cruzada**
- **Ajuste Hiperparâmetros**
- **Boosting**
- **Stacking**


Compartilhar conhecimento é uma alegria—viva ao aprendizado contínuo, boa pratica e bons estudo a quem estiver lendo, abraços!

# Funções, bibliotecas e Dataframe ficticios

In [1]:
RANDOM_STATE = 3141592

In [2]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
import requests

# CARREGANDO BASE
arquivo = 'fake_database'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)
df = fake_database2()
display(df.head())

arquivo = 'funcoes_estatisticas'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)
print('TUDO OK')

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,...,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,4.875997,-1.491729,2.491069,1.874948,-0.000381,1.478758,3.972563,0.230329,5.149503,0.595726,...,sim,não,sim,não,sim,A,B,A,C,D
1,3.750108,-0.510656,-0.462908,0.928715,0.496968,-0.995687,2.219878,-1.951839,0.335021,-0.071709,...,não,não,não,sim,não,B,B,C,C,B
2,1.915908,2.793605,2.989653,1.743696,-0.213394,-0.274895,0.106642,1.605156,2.169961,-1.126328,...,sim,sim,sim,não,sim,C,A,C,B,C
3,3.575351,2.779022,4.928382,3.046386,-1.832086,-2.961492,1.171947,1.044997,3.281876,-1.733729,...,não,não,não,sim,não,B,A,A,D,D
4,4.812294,0.316227,0.704474,0.908321,0.050424,-2.080064,2.597434,-1.643092,-0.138313,-1.498914,...,sim,não,não,não,não,A,B,A,D,B


TUDO OK


In [4]:
# Variavel Dependente
var_dep = 'y'
y = df[var_dep]
display(y.head())

x = df.drop('y', axis=1)
display(x.head())

0    0
1    1
2    0
3    1
4    1
Name: y, dtype: object

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,4.875997,-1.491729,2.491069,1.874948,-0.000381,1.478758,3.972563,0.230329,5.149503,0.595726,sim,não,sim,não,sim,A,B,A,C,D
1,3.750108,-0.510656,-0.462908,0.928715,0.496968,-0.995687,2.219878,-1.951839,0.335021,-0.071709,não,não,não,sim,não,B,B,C,C,B
2,1.915908,2.793605,2.989653,1.743696,-0.213394,-0.274895,0.106642,1.605156,2.169961,-1.126328,sim,sim,sim,não,sim,C,A,C,B,C
3,3.575351,2.779022,4.928382,3.046386,-1.832086,-2.961492,1.171947,1.044997,3.281876,-1.733729,não,não,não,sim,não,B,A,A,D,D
4,4.812294,0.316227,0.704474,0.908321,0.050424,-2.080064,2.597434,-1.643092,-0.138313,-1.498914,sim,não,não,não,não,A,B,A,D,B


In [5]:
# DUMMYRIZAÇÃO
colunas_categoricas = []
colunas_binarias = []
colunas_mais3_categorias = []

for coluna in x.columns:
    if df[coluna].dtype == 'O':
        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_mais3_categorias.append(coluna)

############################################################################################
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_categoricas), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 

############################################################################################
# NORMALIZAÇÃO
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
x = normalizacao.fit_transform(x)

#df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()
#df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

############################################################################################
# DEFININDO A VARIAVEL DEPENDENTE
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

############################################################################################
x_inteiro = x
y_inteiro = y

# DIVIDINDO BASE EM TREINO E TESTE
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = 5) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

print(x_treino.shape, x_teste.shape)
print(y_treino.shape, y_teste.shape)

2 niveis: x_bin0 => ['sim' 'não']
2 niveis: x_bin1 => ['não' 'sim']
2 niveis: x_bin2 => ['sim' 'não']
2 niveis: x_bin3 => ['não' 'sim']
2 niveis: x_bin4 => ['sim' 'não']
3 niveis: x_cat0 => ['A' 'B' 'C']
3 niveis: x_cat1 => ['B' 'A' 'C']
3 niveis: x_cat2 => ['A' 'C' 'B']
3 niveis: x_cat_0 => ['C' 'B' 'D' 'A']
3 niveis: x_cat_1 => ['D' 'B' 'C' 'A']
(750, 32) (250, 32)
(750,) (250,)


In [6]:
# Função para avaliação de modelos exibindo metricas de avaliação
import requests
arquivo = 'ML_supervised_learning'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
response = requests.get(url)
code = response.text
exec(code)

# =======================================

In [7]:
import xgboost as xgb

In [14]:
xg_cl = xgb.XGBClassifier(
    objective='binary:logistic' # é para que o modelo atribua valores de probabilidade para as classes
    )
xg_cl.fit(x_treino, y_treino)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

O parâmetro `objective` no XGBoost define a função de perda (loss function) usada para treinar o modelo, ou seja, ele especifica o tipo de tarefa de aprendizado que você deseja resolver.

No caso de `binary:logistic`, ele é utilizado para **classificação binária**. Esse objetivo pressupõe que o problema envolve duas classes e, portanto, treina o modelo para prever a probabilidade de uma instância pertencer à classe "1" (classe positiva), aplicando uma função logística (sigmoide) para transformar as saídas do modelo em probabilidades no intervalo [0, 1]. O resultado é a probabilidade associada à classe positiva.

Quando você faz a predição com o modelo treinado, ele retorna um valor de probabilidade, e você pode definir um limiar para classificar as instâncias como pertencentes à classe 0 ou 1 (por exemplo, atribuir à classe 1 se a probabilidade for maior que 0,5).

Este é um objetivo comum para tarefas como detecção de fraudes, diagnósticos médicos binários, etc.

In [ ]:
preds = xg_cl.predict(x_teste)
avaliar_modelo(preds, y_teste)

**********************************************************************
Matriz de Confusão:  :

[[119  11]
 [  5 115]]
**************************************************
Relatório de Classificação:  :

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       130
           1       0.91      0.96      0.93       120

    accuracy                           0.94       250
   macro avg       0.94      0.94      0.94       250
weighted avg       0.94      0.94      0.94       250

**********************************************************************


In [ ]:
import matplotlib.pyplot as plt

# Configure o tamanho da figura e a resolução
plt.figure(figsize=(15, 5), dpi=300)  # Ajuste o tamanho e a resolução conforme necessário

# Plotar a árvore
xgb.plot_tree(xg_cl, 
              num_trees=1, # modelo N a ser visualizado
              ax=plt.gca())
plt.show()